In [1]:
%load_ext autoreload
%autoreload 2
import utils.embeddings as em
from utils.metadata import fetch_metadata
from utils.files import load_contents
from utils.learn import evaluate, leave_one_out, repeated_k_fold
from utils.display import plot_report, plot_vs_dummy

In [2]:
import plotly.express as px
import pandas as pd
import datapane as dp
from gensim.models import Doc2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
#from rvfln.rvfln import RVFLNClassifier

In [3]:
semantic = em.load_embeddings("doc2vec")
stylistic = em.load_embeddings("stylistic")

In [5]:
data = fetch_metadata()
data = data[~data.skal_fjernes]
contents = load_contents().dropna()
data = data.merge(contents, on="id_nummer", how="inner")
data = em.add_embeddings(
    data,
    semantic=semantic,
    stylistic=stylistic
)
data = data.dropna(subset="etnicitet")

In [22]:
px.histogram(data.etnicitet)

In [37]:
report = evaluate(
    RandomForestClassifier,
    data=data,
    embeddings="stylistic",
    labels="etnicitet",
    cross_validator=repeated_k_fold(n_folds=5, n_repetitions=30),
    #oversampling=True,
    class_weight="balanced",
)

In [34]:
perf_plot = plot_report(report)
dummy_plot = plot_vs_dummy(report)

In [35]:
perf_plot

In [36]:
dummy_plot